##### This notebook contains all of the subroutines necessary to resolve the interelectrode energy balance. convergence is achieved by a hybrid between a bisection and a customed newtonian method. Convergence is achieved to a level of 99.9 percent. 

In [ ]:
import numpy
from numpy import pi
import sympy
import math
import scipy
import scipy.optimize
from scipy import constants


from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot, cm
%matplotlib inline
#
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

from IPython.core.display import HTML
css_file = './numericalmoocstyle.css'
HTML(open(css_file, "r").read())

In [ ]:
def get_ne(Te_K, e_iz_J, Pt):
    '''
    Parameters
    ----------
    Te_K: Plasma Temperature in Kelvin
    e_iz: Ionization potential of species in Joules
    Pt: Background Pressure
    
    Return:
    ----------
    number density of electrons, ions, and neutrals
    '''
    ne_var = sympy.symbols('ne_var')
    #Saha Eqn
    f_c =  ( ( (2*pi*constants.m_e*constants.k*Te_K) / (constants.h**2) )**(3/2) ) *\
                 math.exp( (-e_iz_J) / (constants.k*Te_K))
    #
    ne_var_eqn = sympy.Eq(0, ((ne_var**2 )/f_c) + (2*ne_var) -\
                          (Pt/(Te_K*constants.k)) )
    #solve for number density:
    nes = sympy.solve(ne_var_eqn, ne_var)
    nos = (nes[1]**2)/f_c
    nis = nes[1]
    #
    return nes[1], nos, nis

In [ ]:
def get_I_ion(Rc, ne, Te_eV, m_i):
    '''
    Parameters
    ----------
    m_i: mass of ions
    Te_eV: Plasma temperature
    ne: number density of electrons
    Rc: cathode radius
    
    Return:
    ----------
    Ion current
    '''
    I_i = pi*(Rc**2)*\
            0.6*(constants.e)*(ne)*numpy.sqrt( (Te_eV*constants.e) / m_i)
        #
    return I_i

In [ ]:
def get_T_cathode(Iarc, root, phi_J, Rc, Iion):
    '''
    Parameters
    ----------
    Iarc: Arc current
    root: Guess for nsolve
    phi: work function
    Rc: cathode radius
    
    Return
    -----------
    Cathode Temperature
    '''
    Tc = sympy.symbols('Tc')
    Tc_eqn = sympy.Eq( Iarc, pi*(Rc**2)*A_R*(Tc**2)*sympy.exp(-phi_J/(constants.k*Tc))\
                  + Iion ) 
    Tcs = sympy.nsolve(Tc_eqn, Tc, root, verify=False)
    Tc_sim = float(Tcs)
    
    return Tc_sim

In [ ]:
def get_Ie_Ith(Rc, phi_J, ne, Te_eV, Ran, Tc):
    '''
    Parameters
    ----------
    Te_eV: Plasma temperature in electron volts
    ne: electron number density
    phi: work function
    Rc: cathode radius
    Ran: anode radius
    Tc: cathode temperature
    
    Return
    ----------
    electron and thermal currents
    '''
    Ie = pi*(Rc**2)*A_R*(Tc**2)*numpy.exp(-phi_J/(constants.k*Tc))
    Ith = 0.25*constants.e*(ne)*\
            (numpy.sqrt((8*constants.e*Te_eV)/(pi*constants.m_e)))*pi*Ran**2
    #
    return Ie, Ith

In [ ]:
def get_anode_voltage_n_flux(Te_eV, Ith, Iarc, Ran, phi_eV):
    '''
    Parameters
    ----------
    Te_eV: Plasma temperature
    I_th: thermal current
    Iarc: Input arc current
    
    Return:
    ----------
    Anode voltage drop in eV, and Heat flux density at anode
    '''
    #
    ua_ev =  - Te_eV * math.log(Ith/Iarc)
    Qan = (Iarc/(pi*Ran**2))*((2.*Te_eV + ua_ev + phi_eV))
    
    return ua_ev, Qan

In [ ]:
def get_Ta_ABL(m_i, root, hv, cp, qa, Ti, Ran, A, B):
    '''
    Parameters
    ----------
    m_i: ion mass
    root: root for nsolve
    hv: heat of vaporization
    cp: specific hear
    qa: heat flux density at anode
    Ti: initial temperature
    Ran: anode temperature
    A,B: constants for vapor pressure function
    '''

    Ta = sympy.symbols('Ta')
    #
    Ta_eqn = qa - ( (0.133*sympy.exp(2.3*(A - B/Ta)))*\
                              sympy.sqrt(m_i/(2*pi*constants.k*Ta)))*\
                   (cp*Ta + hv+ cp*(Ta-Ti))
    #
    Ta_sol = sympy.nsolve(Ta_eqn, root, verify=False)
    #
    Tan = float(Ta_sol)
    
    ablation = (0.133*numpy.exp(2.3*(A - B/Tan)))*\
                numpy.sqrt(m_i/(2*pi*constants.k*Tan))*\
                numpy.pi*(Ran**2)*1e6
    #
    return Tan, ablation

In [ ]:
def get_collisions_sigma(ne,ni,no, Te_K):
    # Parameters for Collisions
    re = 2.8179403267e-15 # classical electron radius in meters
    gamma = numpy.exp(0.577) #from Euler's constant
    #------------
    Qm = 40e-20


    ke = math.sqrt((4*pi*ne*(constants.e**2))/(Te_K*constants.k))

    lnA = numpy.log((4*constants.k*Te_K)/((gamma**2)*(ke)*(constants.e**2))) -\
        2*numpy.log(numpy.sqrt(2))

    # electron-neutral collision frequency
    v_eo = (4/3)*numpy.sqrt((8*constants.k*Te_K)/(pi*constants.m_e))*\
                Qm*(no)

    # electron-neutral collision frequency
    v_ei = (4/3)*numpy.sqrt(2*pi)*(ni)*\
            (((constants.e**2)/(constants.k*Te_K))**2)*\
            numpy.sqrt((constants.k*Te_K)/constants.m_e)*lnA
    #total electron collision frequency
    ve_t = v_ei + v_eo
    #
    #electrical conductivity
    sigma = (ne*constants.e**2)/(constants.m_e* ve_t)
    #
    return ve_t, sigma

In [ ]:
def get_Upl(Iarc, Lgap, sigma, Rarc):
    #
    u_pl = (Iarc*Lgap)/(sigma*pi*Rarc**2)
    return u_pl

In [ ]:
def get_fluxes(Tc, Ti, lmbda, Rc, Em):
    #
    # thermal conduction flux (W/m2)
    Q_con = ((Tc-Ti)*lmbda)/(Rc*(pi**(3/2)) )
    # radiative  flux (W/m2)
    Q_rad = Em*constants.sigma*(Tc**4)
    #
    return Q_con, Q_rad

In [ ]:
def get_U_cathode(Rc, Iion, qcon, qrad, phi_eV, e_iz_eV):
    #
    Uc = ((pi*(Rc**2))/(Iion))*( qcon + qrad ) + phi_eV + e_iz_eV
    #
    return Uc

In [ ]:
def get_energy_conservation(Iarc, Upl, Uc, Ie, Ta, Te_eV, Ua_eV, e_iz_eV, Iion,\
                           m_i, ne, Lgap, Ran, ve):
    #
    #Joule Heating
    JH = Iarc*(Upl) + Uc*Ie
    #
    #Losses
    Tan_eV = Ta/(11604.52500617)
    #
    loss12 = Iarc*(2*Te_eV + Ua_eV) + e_iz_eV*Iion
    loss3 = 3*(constants.m_e/m_i)*\
            (Te_eV-Tan_eV)*ne*Lgap*pi*(Ran**2)*ve*constants.e
    # Conservation:
    cons = JH/(loss12 + loss3)
    #Error:
    err = numpy.abs(1 - cons)
    #
    return err
    

In [ ]:
def get_energy_balance(Te_eV, Pt, Ti, Iarc, Rc, Ran, Lgap, e_iz_eV, m_i, phi_eV,\
                      h_v, c_p, A, B, Em, root, lmbda):
    '''
    Parameters
    ----------
    Te_eV: Plasma temperature
    Pt: Background Pressure
    Ti: initial temperature
    Iarc: Input current
    Rc: cathode radius
    Ran: anode radius
    Lgap: gap between anode and cathode
    e_iz: ionization potential
    phi_eV: work function of species #in electron volts
    '''
    #
    Te_K = Te_eV*(11604.52500617) # K
    e_iz_J = e_iz_eV*constants.e 
    phi_J = phi_eV*constants.e
    #
    #Define Arc Radius:
    if (Iarc < 50.0):
        Rarc = Ran
    if (Iarc >= 50.0):
        Rarc = Ran*0.02*Iarc
    #------

    # Get Electron, Ion, and Neutral number density:
    ne, no, ni = get_ne(Te_K, e_iz_J, Pt)
    #------

    # Ion current:
    I_ion = get_I_ion(Rc, ne, Te_eV, m_i)
    #------

    #Cathode Temperature:
    T_c = get_T_cathode(Iarc, root, phi_J, Rc, I_ion)
    #-------

    # Electron and Thermal Currents:
    I_e, I_th = get_Ie_Ith(Rc, phi_J, ne, Te_eV, Ran, T_c)
    #--------

    # Anode Voltage Drop and Heat flux density at anode:
    Ua_eV, q_a =  get_anode_voltage_n_flux(Te_eV, I_th, Iarc, Ran, phi_eV) #eV
    #---------

    # Anode Temperature and Ablation:
    T_a, abl = get_Ta_ABL(m_i, root, h_v, c_p, q_a, Ti, Ran, A, B)
    #---------
    
    #Collisions and electrical conductivity
    ve, sigma_e = get_collisions_sigma(ne,ni,no, Te_K)
    #---------
    
    # Inter-electrode Voltage
    Upl = get_Upl(Iarc, Lgap, sigma_e, Rarc)
    #---------
    
    #fluxes (thermal and radiation)
    q_con, q_rad = get_fluxes(T_c, Ti, lmbda, Rc, Em)
    
    #Cathode voltage:
    U_c = get_U_cathode(Rc, I_ion, q_con, q_rad, phi_eV, e_iz_eV)
    
    #Energy conservation
    Error = get_energy_conservation(Iarc, Upl, U_c, I_e, T_a, Te_eV, Ua_eV, e_iz_eV,\
                                    I_ion, m_i, ne, Lgap, Ran, ve)
    #
    # Return resolved variables:
    return ne, no, ni, I_ion, T_c, I_e, Ua_eV, q_a, T_a, abl, sigma_e, Upl, U_c,\
            Error
    

### Parameters

In [ ]:
# Inputs to Model

#Plasma Temperature guess
#TeeV = 0.55 # given in eV

#Background Pressure
Pt = 300 * 133.322 # torr to Pa

#Initial Cathode and Anode Temperatures, assume room temp in Kelvin:
To = 298.15

#Input Arc current
Iarc = 20.

#dimensions
Rc = Rc = (12.5/2.) * 1./1000. # cathode radius [meters]
Ran = 6.35/2. * 1./1000.  #anode radius [meters]
Lgap = 3.5 * 1./1000. #interelectrode gap [meters]

# Ion Mass:
UAMU = 1.660538921e-27 # Unified atomic mass unit in Kg

mi_He = UAMU*4.002602
mi_C = UAMU*12.0107

m_i_AMU = (mi_He + mi_C) # given in kilograms

#Richardson Constant:
A_R = 1.2e6 #constant in electron current density eqn [A/(m^2 K^2)], pg.62

# Carbon-Specific Properties
phi_C_eV = 4.81 #eV # work function
dH_C = (355.8/(0.012))*1000. #convert kJ/Kmol to J/kg, heat of vaporization carbon
cp_C = 710.0 # J/(Kg*K) #specific heat @ constant pressure
A_C = 15.73
B_C = 40030.0
Em_C = 0.70

#Thermal Conductivity Carbon
lmbda_C = 1.7 #thermal conductivity carbon W m-1 K-1

E_C_eV = 11.2603 # ionization energy of carbon in eV
E_C_J = 11.2603*constants.e # ionization energy of carbon in Joules

In [ ]:
# n_e, n_o, n_i, Iion, Tc,\
# Ie, UaeV, qa, Ta, ABL,\
# sigma, U_pl, Uc, cons_err = get_energy_balance(Te_eV, Pt, To, Iarc, Rc, Ran,\
#                                                Lgap, E_C_eV, m_i_AMU, phi_C_eV,\
#                                                   dH_C, cp_C, A_C, B_C, Em_C,\
#                                                     3000., lmbda_C)

In [ ]:
ta_int = 0.5 #Lower Range for bisection method
tb_int = 0.75 #Upper Range for bisection method
rs = 3000. #for root finding

In [ ]:
def bisection(Ta_int,Tc_int,\
              Pt, Ti, Iarc, Rc, Ran, Lgap, e_iz_eV, m_i, phi_eV,\
                      h_v, c_p, A, B, Em, root, lmbda):
    '''
    Solves the energy balance equation using the bisection method to within 95 percent
    agreement between joule heating and losses
    
    Parameters
    ----------
    
    a_int,b_int - range of plasma temperatures

    Pt: Background Pressure
    Ti: initial temperature
    Iarc: Input current
    Rc: cathode radius
    Ran: anode radius
    Lgap: gap between anode and cathode
    e_iz: ionization potential
    phi_eV: work function of species #in electron volts
    '''
    #Limit the number of iterations
    itermax = 1000
    itermax2 = 1000
    iterations = 0 
    #
    #Tolerance for energy conservation (95%)
    tol = 5e-2
    tol2 = 1e-3
    #
    # intiate midpoint:
    Tb_int = (numpy.abs(Tc_int-Ta_int)/2.0) + Ta_int
    #
    cons_err =  1.1 + tol
    #
    while (cons_err > tol) :
        #and (iterations > itermax)
        #
        # At end point a:
        ne_a, no_a, ni_a, Iion_a, Tc_a,\
         Ie_a, UaeV_a, qa_a, Ta_a, ABL_a,\
            sigma_a, U_pl_a, Uc_a, cons_err_a =\
                    get_energy_balance(Ta_int, Pt, Ti, Iarc, Rc, Ran, Lgap, e_iz_eV,\
                                                            m_i, phi_eV,h_v,c_p,A,B,\
                                                                   Em,root, lmbda)
        # At end point c:
        ne_c, no_c, ni_c, Iion_c, Tc_c,\
         Ie_c, UaeV_c, qa_c, Ta_c, ABL_c,\
            sigma_c, U_pl_c, Uc_c, cons_err_c =\
                    get_energy_balance(Tc_int, Pt, Ti, Iarc, Rc, Ran, Lgap, e_iz_eV,\
                                                            m_i, phi_eV,h_v,c_p,A,B,\
                                                                   Em,root, lmbda)
        # At midpoint b_int:
        ne_b, no_b, ni_b, Iion_b, Tc_b,\
         Ie_b, UaeV_b, qa_b, Ta_b, ABL_b,\
            sigma_b, U_pl_b, Uc_b, cons_err_b =\
                get_energy_balance(Tb_int, Pt, Ti, Iarc,Rc, Ran, Lgap, e_iz_eV,\
                                                            m_i, phi_eV,h_v,c_p,A,B,\
                                                                Em,root, lmbda)
        #
#         print('hello, iteration no.: %.4g' %iterations)
        
#         print('err_a %.4g ' %cons_err_a)
#         print('err_b %.4g ' %cons_err_b)
#         print('err_c %.4g ' %cons_err_c)
        
        #------------------
        x = float('inf') # using for logic statements
#         print (cons_err_b == x)
        #------------------

        if (cons_err_a==x) or (cons_err_b==x) or (cons_err_c==x):
#             print('Ta: %.4g ' % Ta_int)
#             print('Tb: %.4g ' %tb_int)
            
#             print('err_a %.4g ' %cons_err_a)
#             print('err_b %.4g ' %cons_err_b)
#             print('err_c %.4g ' %cons_err_c)
        #
            cons_err = min( cons_err_a, cons_err_b, cons_err_c )
            #
            if (cons_err==cons_err_a):
                T_plasma = Ta_int
            if (cons_err==cons_err_b):
                T_plasma = Tb_int
            #
            #
            if (T_plasma==Tb_int):
                #
                iterations2 = 0
            #
                while (cons_err > tol):
            
                    T_plasma -= 1e-4
    
                    ne_b, no_b, ni_b, Iion_b, Tc_b,\
                     Ie_b, UaeV_b, qa_b, Ta_b, ABL_b,\
                        sigma_b, U_pl_b, Uc_b, cons_err_b =\
                                get_energy_balance(T_plasma, Pt, Ti, Iarc, Rc,\
                                                   Ran, Lgap, e_iz_eV, m_i, phi_eV,
                                                   h_v,c_p,A,B, Em,root, lmbda)

                    cons_err = cons_err_b 
                    print('#1: error %.4g' %cons_err)
                    iterations2 += 1
                    
                    if (iterations2>itermax):
                        print('maximum iterations exceeded 1')
                        break
            #
                    if (cons_err < tol):
                        print('#1: 5e-2 convergence after bisection caused inf val.')
                        break
    
            if (T_plasma == Ta_int):
                #
                print('at this point lower Plasma_temp : %.4g ' %T_plasma)
                iterations2 = 0
                #
                while (cons_err > tol):
            
                    T_plasma += 1e-4
    
                    ne_a, no_a, ni_a, Iion_a, Tc_a,\
                         Ie_a, UaeV_a, qa_a, Ta_a, ABL_a,\
                            sigma_a, U_pl_a, Uc_a, cons_err_a =\
                                    get_energy_balance(T_plasma, Pt, Ti, Iarc, Rc,\
                                                    Ran, Lgap, e_iz_eV, m_i, phi_eV,
                                                       h_v,c_p,A,B, Em,root, lmbda)
    
                    cons_err = cons_err_a 
                    print('#2: error %.4g' %cons_err)
                    iterations2 += 1
            #
                    if (iterations2>itermax):
                        print('maximum iterations exceeded 2')
                        break
            #
                    if (cons_err < tol):
                        print('#2: 5e-2 convergence after bisection caused inf val.')
                        break
        
        #------------------
        # Bisection Action:
        # 
        else:
            if (cons_err_a != x) and (cons_err_b!= x) and (cons_err_c!= x) :
                cons_err = min(cons_err_a, cons_err_b, cons_err_c)

                if (cons_err == cons_err_a):
                    T_plasma = Ta_int
                elif (cons_err == cons_err_b):
                    T_plasma = Tb_int
                elif (cons_err == cons_err_c):
                    T_plasma = Tc_int

                # Check for convergence:
                if (cons_err < tol ):
                    print('95 percent convergence achieved!')
                    break
                #
                diff_err_ab = numpy.abs(cons_err_b - cons_err_a)
                diff_err_bc = numpy.abs(cons_err_c - cons_err_b)
                
#                 print('error A-B:%.4g' %diff_err_ab)
#                 print('error B-C:%.4g' %diff_err_bc)

                if (diff_err_ab < diff_err_bc):
                    cons_err = min(cons_err_a, cons_err_b)
                    Tc_int = Tb_int
                    Tb_int = (numpy.abs(Tc_int-Ta_int)/2.0) + Ta_int
                    #
#                     print('Tc:%.4g' %Tc_int)
#                     print('Tb:%.4g' %Tb_int)
                #
                elif (diff_err_bc < diff_err_ab):
                    cons_err = min(cons_err_b, cons_err_c)
                    Ta_int = Tb_int
                    Tb_int = (numpy.abs(Tc_int-Ta_int)/2.0) + Ta_int

                iterations +=1
                iterations2 = iterations

                if (iterations>itermax):
                    print('maximum iterations exceeded 3')
                    break

    return T_plasma, iterations, iterations2, cons_err


In [ ]:
T_test,it1,it2,error = bisection(ta_int, tb_int, Pt, To, Iarc, Rc, Ran,\
                                               Lgap, E_C_eV, m_i_AMU, phi_C_eV,\
                                                  dH_C, cp_C, A_C, B_C, Em_C,\
                                                    rs, lmbda_C)

In [ ]:
it2

In [ ]:
T_test

In [ ]:
error

In [ ]:
def converge_custom(Tplasma, error_b, Pt, Ti, Iarc, Rc,\
                    Ran, Lgap, e_iz_eV, m_i, phi_eV,\
                      h_v, c_p, A, B, Em, root, lmbda):
    
    #Parameters Tplasma, error_b from the bisection exit
    #
    tolerance = 1e-3
    #
    error = error_b
    #
    iterations = 0
    #
    T = Tplasma
    #
    itermax = 100.
    #
    error_ref = error
    #
    while error > tolerance:
        #
        T += 1e-6
        #
        ne_a, no_a, ni_a, Iion_a, Tc_a,\
         Ie_a, UaeV_a, qa_a, Ta_a, ABL_a,\
            sigma_a, U_pl_a, Uc_a, cons_err_a =\
                    get_energy_balance(T, Pt, Ti, Iarc, Rc, Ran, Lgap, e_iz_eV,\
                                                        m_i, phi_eV,h_v,c_p,A,B,\
                                                                   Em,root, lmbda)
        
        if (cons_err_a > error_ref) or (cons_err_a == error_ref):
            print('wrong direction (sign) or try different add/subtract from original value')
            break
        #
        error = cons_err_a
        print(error)
        #
        iterations += 1
        #
        if (error < tolerance):
            print('convergence achieved to within 1e-3')
            break
            
        if (iterations>itermax):
            print('maximum iterations exceeded')
            break
        #
    return T, iterations

In [ ]:
def converge_slow(Tplasma, Pt, Ti, Iarc, Rc,\
                    Ran, Lgap, e_iz_eV, m_i, phi_eV,\
                      h_v, c_p, A, B, Em, root, lmbda):
    
    #Start on lower end and move up until convergence, this will be slow
    #
    tolerance = 1e-3
    #
    iterations = 0
    #
    T = Tplasma
    #
    itermax = 50000.
    #
    error = 1.1 + tolerance
    #
    while error > tolerance:
        #
        T += 1e-5
        #
        ne_a, no_a, ni_a, Iion_a, Tc_a,\
         Ie_a, UaeV_a, qa_a, Ta_a, ABL_a,\
            sigma_a, U_pl_a, Uc_a, cons_err_a =\
                    get_energy_balance(T, Pt, Ti, Iarc, Rc, Ran, Lgap, e_iz_eV,\
                                                        m_i, phi_eV,h_v,c_p,A,B,\
                                                                   Em,root, lmbda)
        
#         if (cons_err_a > error) or (cons_err_a == error):
#             print('wrong direction (sign) or try different add/subtract from original value')
#             break
        #
        error = cons_err_a
        print(error)
        #
        iterations += 1
        #
        if (error < tolerance):
            print('convergence achieved to within 1e-3')
            break
            
        if (iterations>itermax):
            print('maximum iterations exceeded')
            break
        #
    return T, iterations

In [ ]:
Tp, it = converge_slow(0.52, Pt, To, Iarc, Rc, Ran,\
                         Lgap, E_C_eV, m_i_AMU, phi_C_eV,\
                            dH_C, cp_C, A_C, B_C, Em_C,\
                                                    rs, lmbda_C)

In [ ]:
Tp

In [ ]:
it

## Summary of Convergence

### 20 amps, 300 torr: 
#### Te = 0.551819999999992

### 40 amps, 300 torr: 
#### Te = 0.5885799999999872

### 50 amps, 300 torr: 
#### Te = 0.63000399999999956

### 50 amps, 300 torr (BETTER SOLUTION for 50 A): 
#### 0.60147999999962926

### 60 amps, 300 torr: 
#### Te = 0.60771000000000541

### 80 Amps, 300 Torr: 
#### Te = 0.61915062499999995

### 100 Amps, 300 Torr: 
#### Te = 0.62941999999999543